In [1]:
pip install spacy


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

     ---------------------------------------- 12.3/12.3 MB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 896.1 kB/s eta 0:00:00
     ---------------------------------------- 94.7/94.7 kB 1.1 MB/s eta 0:00:00
     --------------------------------------- 45.9/45.9 kB 34.0 kB/s eta 0:00:00
     -------------------------------------- 48.9/48.9 kB 488.9 kB/s eta 0:00:00
     -------------------------------------- 181.6/181.6 kB 1.6 MB/s eta 0:00:00
     -------------------------------------- 480.9/480.9 kB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 7.0/7.0 MB 1.8 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [6]:
import csv
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('maxent_ne_chunker', quiet=True)
nltk.download('words', quiet=True)

# Load the dataset from the CSV file
dataset = []
with open('connectivity.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        dataset.append(row['user_query'])

# Tokenization
tokenized_dataset = [word_tokenize(query) for query in dataset]

# Part-of-Speech Tagging
pos_tagged_dataset = [pos_tag(tokens) for tokens in tokenized_dataset]

# Named Entity Recognition
ner_dataset = []
for pos_tags in pos_tagged_dataset:
    chunked = ne_chunk(pos_tags)
    entities = []
    for entity in chunked:
        if hasattr(entity, 'label'):
            entities.append((entity.label(), ' '.join([word for word, _ in entity])))
    ner_dataset.append(entities)

# Print the results
print("Tokenized Dataset:")
for tokens in tokenized_dataset:
    print(tokens)

print("\nPart-of-Speech Tagged Dataset:")
for pos_tags in pos_tagged_dataset:
    print(pos_tags)

print("\nNamed Entity Recognition (NER) Dataset:")
for ner_tags in ner_dataset:
    print(ner_tags)


Tokenized Dataset:
['I', 'ca', "n't", 'connect', 'to', 'the', 'internet', '.']
['-', 'Check', 'if', 'other', 'devices', 'on', 'the', 'network', 'are', 'able', 'to', 'connect', '.']
['-', 'Restart', 'your', 'modem/router', 'and', 'try', 'again', '.']
['-', 'Verify', 'that', 'your', 'Wi-Fi', 'password', 'is', 'correct', '.']
['-', 'Disable', 'any', 'VPN', 'or', 'proxy', 'settings', 'on', 'your', 'device', '.']
['-', 'Contact', 'your', 'internet', 'service', 'provider', 'if', 'the', 'issue', 'persists', '.']
['My', 'Wi-Fi', 'signal', 'is', 'weak', '.']
['-', 'Move', 'closer', 'to', 'your', 'Wi-Fi', 'router', 'to', 'improve', 'signal', 'strength', '.']
['-', 'Remove', 'any', 'physical', 'obstructions', 'blocking', 'the', 'Wi-Fi', 'signal', '.']
['-', 'Consider', 'using', 'a', 'Wi-Fi', 'range', 'extender', 'or', 'access', 'point', 'for', 'better', 'coverage', '.']
['-', 'Update', 'your', 'Wi-Fi', 'router', 'firmware', 'to', 'the', 'latest', 'version', '.']
['-', 'Check', 'for', 'interferenc

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Prepare the dataset
# Prepare the dataset
queries = [
    "I can't connect to the internet.",
    "- Check if other devices on the network are able to connect.",
    "- Restart your modem/router and try again.",
    "- Verify that your Wi-Fi password is correct.",
    "- Disable any VPN or proxy settings on your device.",
    "My Wi-Fi signal is weak.",
    "- Move closer to your Wi-Fi router to improve signal strength.",
    "- Remove any physical obstructions blocking the Wi-Fi signal.",
    "- Consider using a Wi-Fi range extender or access point for better coverage.",
    "- Update your Wi-Fi router firmware to the latest version.",
    "- Check for interference from other electronic devices.",
    "I'm getting a 'No internet access' error.",
    "- Ensure your device is connected to the correct Wi-Fi network.",
    "- Restart your modem/router and wait for it to fully boot up.",
    "- Check if there are any known network outages in your area.",
    "- Update the network adapter drivers on your device.",
    "- Temporarily disable any firewall or antivirus software and test the connection.",
    "I can't access a specific website.",
    "- Clear your browser cache and cookies.",
    "- Try accessing the website using a different browser.",
    "I'm having trouble connecting to my work VPN.",
    "- Double-check your VPN credentials and authentication settings.",
    "- Ensure you have a stable internet connection before connecting.",
    "- Verify that your firewall settings allow VPN connections.",
    "- Try connecting to the VPN from a different network or location."
]

intent_labels = [
    "Wi-Fi Network Connection",
    "Wi-Fi Network Connection",
    "Wi-Fi Network Connection",
    "Wi-Fi Network Connection",
    "Wi-Fi Network Connection",
    "Wi-Fi Signal",
    "Wi-Fi Signal",
    "Wi-Fi Signal",
    "Wi-Fi Signal",
    "Wi-Fi Signal",
    "Wi-Fi Signal",
    "No Internet Access",
    "No Internet Access",
    "No Internet Access",
    "No Internet Access",
    "No Internet Access",
    "No Internet Access",
    "Website Access",
    "Website Access",
    "Website Access",
    "Work VPN Connection",
    "Work VPN Connection",
    "Work VPN Connection",
    "Work VPN Connection",
    "Work VPN Connection"
]


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(queries, intent_labels, test_size=0.2, random_state=42)

# Vectorize the text data using CountVectorizer
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Train a logistic regression model
classifier = LogisticRegression(multi_class="auto")
classifier.fit(X_train_vectorized, y_train)

# Predict intents for test data
y_pred = classifier.predict(X_test_vectorized)

# Evaluate the model
accuracy = (y_pred == y_test).mean()
print(f"Accuracy: {accuracy}")


Accuracy: 0.2


In [13]:
import nltk
from sklearn.svm import SVC
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Prepare feature sets
featuresets = []
for example, label in zip(dataset, intent_labels):
    features = {word.lower(): True for word in example[0].split()}
    featuresets.append((features, label))

# Shuffle the dataset
random.shuffle(featuresets)

# Split dataset into training and testing
train_set = featuresets[:int(0.8 * len(featuresets))]
test_set = featuresets[int(0.8 * len(featuresets)):]

# Define the classifier pipeline
classifier = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('svm', SVC(kernel='linear'))
])

# Train the classifier
classifier.fit([x for x, _ in train_set], [y for _, y in train_set])

# Evaluate the classifier
train_predictions = classifier.predict([x for x, _ in train_set])
train_accuracy = accuracy_score([y for _, y in train_set], train_predictions)

test_predictions = classifier.predict([x for x, _ in test_set])
test_accuracy = accuracy_score([y for _, y in test_set], test_predictions)

print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)


Training Accuracy: 0.35
Testing Accuracy: 0.0


In [16]:
knowledge_base = {
    "Wi-Fi Network Connection": [
        "I can't connect to the internet.",
        "- Check if other devices on the network are able to connect.",
        "- Restart your modem/router and try again.",
        "- Verify that your Wi-Fi password is correct.",
        "- Disable any VPN or proxy settings on your device.",
        "- Contact your internet service provider if the issue persists."
    ],
    "Network Connectivity": [
        "My Wi-Fi signal is weak.",
        "- Move closer to your Wi-Fi router to improve signal strength.",
        "- Remove any physical obstructions blocking the Wi-Fi signal.",
        "- Consider using a Wi-Fi range extender or access point for better coverage.",
        "- Update your Wi-Fi router firmware to the latest version.",
        "- Check for interference from other electronic devices."
    ],
    "No Internet Access": [
        "I'm getting a 'No internet access' error.",
        "- Ensure your device is connected to the correct Wi-Fi network.",
        "- Restart your modem/router and wait for it to fully boot up.",
        "- Check if there are any known network outages in your area.",
        "- Update the network adapter drivers on your device.",
        "- Temporarily disable any firewall or antivirus software and test the connection."
    ],
    "Website Access": [
        "I can't access a specific website.",
        "- Clear your browser cache and cookies.",
        "- Try accessing the website using a different browser.",
        "- Check if the website is down by using an online service.",
        "- Disable any browser extensions that might be causing conflicts.",
        "- Restart your device and try accessing the website again."
    ],
    "Work VPN Connection": [
        "I'm having trouble connecting to my work VPN.",
        "- Double-check your VPN credentials and authentication settings.",
        "- Ensure you have a stable internet connection before connecting.",
        "- Verify that your firewall settings allow VPN connections.",
        "- Try connecting to the VPN from a different network or location.",
        "- Contact your IT department for further assistance."
    ]
    # Add more intent categories and associated queries here
}

def search_solution(user_query, intent):
    matching_solutions = []
    
    if intent in knowledge_base:
        matching_solutions = knowledge_base[intent]
    
    return matching_solutions

def generate_response(solutions):
    response = ""
    
    if len(solutions) > 0:
        response += "Solution: To troubleshoot your issue, please follow these steps:\n"
        response += "\n".join(solutions)
    else:
        response = "I'm sorry, but I couldn't find a solution for your issue."
    
    return response

# Example usage
user_query = "I can't connect to the internet."
identified_intent = "Wi-Fi Network Connection"

# Search for relevant solutions
matching_solutions = search_solution(user_query, identified_intent)

# Generate response based on the retrieved solutions
response = generate_response(matching_solutions)

print(response)

Solution: To troubleshoot your issue, please follow these steps:
I can't connect to the internet.
- Check if other devices on the network are able to connect.
- Restart your modem/router and try again.
- Verify that your Wi-Fi password is correct.
- Disable any VPN or proxy settings on your device.
- Contact your internet service provider if the issue persists.
